# Анализ базы данных сервиса для чтения книг

Компания приобрела крупный сервис для чтения книг по подписке. Необходимо проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Введение

**Проект направлен** на исследование базы данных нового сервиса для дальнейшей разработки кампании по его развитию.

**Задачи исследования**:

* Посчитать, сколько книг вышло после 1 января 2000 года;
* Посчитать количество обзоров и среднюю оценку для каждой книги;
* Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — исключить из анализа брошюры;
* Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
* Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

**Описание исходных данных:**

Таблица *books* - содержит данные о книгах:
* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.

Таблица *authors* - содержит данные об авторах:
* author_id — идентификатор автора;
* author — имя автора.

Таблица *publishers* - содержит данные об издательствах:
* publisher_id — идентификатор издательства;
* publisher — название издательства.

Таблица *ratings* - содержит данные о пользовательских оценках книг:
* rating_id — идентификатор оценки;
* book_id — идентификатор книги;
* username — имя пользователя, оставившего оценку;
* rating — оценка книги.

Таблица *reviews* - содержит данные о пользовательских обзорах:
* review_id — идентификатор обзора;
* book_id — идентификатор книги;
* username — имя автора обзора;
* text — текст обзора.

## Обзор данных

In [1]:
#импортируем необходимые библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
#записываем переменную с параметрами подключения к базе данных - данные для подключения скрыты
connection_string = \
'*://*:*:*/*'

#записываем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

con=engine.connect()

Познакомимся с базой данных, посмотрим на каждую таблицу: 

### Таблица books

In [3]:
query = '''
SELECT *
FROM books
LIMIT 5
'''

pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = '''
SELECT *
FROM books
'''

display(pd.io.sql.read_sql(query, con = engine).info())
pd.io.sql.read_sql(query, con = engine)['num_pages'].describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


None

count    1000.00000
mean      389.11100
std       229.39014
min        14.00000
25%       249.00000
50%       352.00000
75%       453.00000
max      2690.00000
Name: num_pages, dtype: float64

### Таблица authors

In [5]:
query = '''
SELECT *
FROM authors
LIMIT 5
'''

pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query = '''
SELECT *
FROM authors
'''

pd.io.sql.read_sql(query, con = engine).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


### Таблица publishers

In [7]:
query = '''
SELECT *
FROM publishers
LIMIT 5
'''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
query = '''
SELECT *
FROM publishers
'''

pd.io.sql.read_sql(query, con = engine).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


### Таблица ratings

In [9]:
query = '''
SELECT *
FROM ratings
LIMIT 5
'''

pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [10]:
query = '''
SELECT *
FROM ratings
'''

display(pd.io.sql.read_sql(query, con = engine).info())
pd.io.sql.read_sql(query, con = engine)['rating'].describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


None

count    6456.000000
mean        3.928284
std         0.943303
min         1.000000
25%         3.000000
50%         4.000000
75%         5.000000
max         5.000000
Name: rating, dtype: float64

### Таблица reviews

In [11]:
query = '''
SELECT *
FROM reviews
LIMIT 5
'''

pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [12]:
query = '''
SELECT *
FROM reviews
'''

pd.io.sql.read_sql(query, con = engine).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


## Исследование данных

### Количество вышедших книг после 1 января 2000 года

In [13]:
query = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'
'''

pd.io.sql.read_sql(query, con = engine)

,count
0,819


Всего вышло 819 книг после 01.01.2000. 

### Количество обзоров и средняя оценка каждой книги

In [14]:
query = '''
SELECT b.book_id,
       b.title,
       COUNT(DISTINCT re.review_id) AS review_count,
       AVG(ra.rating) AS avg_rating
FROM books AS b
LEFT JOIN reviews AS re ON b.book_id = re.book_id
LEFT JOIN ratings AS ra ON b.book_id = ra.book_id
GROUP BY b.book_id
ORDER BY review_count DESC, b.title DESC
'''

pd.io.sql.read_sql(query, con = engine)

,book_id,title,review_count,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,854,The Road,6,3.772727
3,779,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
4,750,The Hobbit or There and Back Again,6,4.125000
...,...,...,...,...
995,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
996,387,Leonardo's Notebooks,0,4.000000
997,221,Essential Tales and Poems,0,4.000000
998,191,Disney's Beauty and the Beast (A Little Golden...,0,4.000000


Видим, что разброс количества уникальных обзоров на книги от 0 до 7.

Средний рейтинг некоторых книг представлен целым числом, что может свидетельствовать о малом количестве оценок (1) или об единодушии читателей. 

Всего в таблице перечислена 1000 книг.

### Издательство, выпустившее наибольшее число книг толще 50 страниц

In [15]:
query = '''
SELECT b.publisher_id,
       p.publisher,
       COUNT(b.book_id) AS books_count
FROM books AS b
LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY b.publisher_id, p.publisher
ORDER BY books_count DESC, p.publisher DESC
LIMIT 1
'''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher,books_count
0,212,Penguin Books,42


Издательство, которое выпустило наибольшее число книг с более, чем 50 страницами — Penguin Books (42 книги).

### Автор с самой высокой средней оценкой книг (только книги с 50 и более оценками)

In [16]:
query = '''
WITH cte1 AS (SELECT book_id
              FROM ratings
              GROUP BY book_id
              HAVING COUNT(DISTINCT rating_id) > 50)

SELECT b.author_id,
       a.author,
       AVG(ra.rating) AS avg_rating
FROM cte1
JOIN books AS b ON cte1.book_id = b.book_id
JOIN authors AS a ON b.author_id = a.author_id
JOIN ratings AS ra ON cte1.book_id = ra.book_id
GROUP BY b.author_id, a.author
ORDER BY avg_rating DESC, a.author
LIMIT 1
'''

pd.io.sql.read_sql(query, con = engine)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097


Автором с самой высокой средней оценкой книг (более 50 оценок) является Джоан Роулинг вместе с Мари Гранпрэ, средний рейтинг их книг составил 4.3. 

### Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [17]:
query = '''
WITH cte1 AS (SELECT DISTINCT username
              FROM ratings
              GROUP BY username
              HAVING COUNT(DISTINCT rating_id) > 48),
     cte2 AS (SELECT cte1.username,
              COUNT(re.review_id) AS avg_review_count
              FROM cte1
              LEFT JOIN reviews AS re ON cte1.username = re.username
              GROUP BY cte1.username)
              
SELECT AVG(cte2.avg_review_count) AS avg_review_count_per_user
FROM cte2
'''

pd.io.sql.read_sql(query, con = engine)

,avg_review_count_per_user
0,24.0


В среднем пользователи с более, чем 48 оценками, пишут 24 обзора. 

## Общий вывод

**Проведен обзор данных:**

* Таблица books: 1000 строк, пропусков в данных нет, диапазон значений столбца num_pages 14-2690;
* Таблица authors: 636 строк, пропусков в данных нет;
* Таблица publishers: 340 строк, пропусков в данных нет;
* Таблица ratings: 6456 строк, пропусков в данных нет, диапазон значений столбца rating 1-5;
* Таблица reviews: 2793 строк, пропусков в данных нет;

**Исследовано:**

* Всего вышло 819 книг после 01.01.2000;
* Разброс количества уникальных обзоров на книги от 0 до 7, средний рейтинг некоторых книг представлен целым числом, что может свидетельствовать о малом количестве оценок (1) или об единодушии читателей;
* Издательство, которое выпустило наибольшее число книг с более, чем 50 страницами — Penguin Books (42 книги);
* Автором с самой высокой средней оценкой книг (более 50 оценок) является Джоан Роулинг вместе с Мари Гранпрэ, средний рейтинг их книг составил 4.3;
* В среднем пользователи с более, чем 48 оценками, пишут 24 обзора.